In [1]:
import pandas as pd
import numpy as np
import requests
from pandas.tseries.offsets import BDay
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def calculate_daily_return(df):
    # Use pct_change() to calculate the percentage change in 'c' (close prices)
    df['daily_return'] = df['c'].pct_change()
    df['abs_daily_return'] = df['daily_return'].abs()
    return df

def get_top_outliers(df, n=10):
    # # Filter for positive returns with 'c' > 0.95 and negative returns with 'c' < 0.8
    # positive_condition = (df['c'] > 0.95) & (df['daily_return'] > 0)
    # negative_condition = (df['c'] < 0.8) & (df['daily_return'] < 0)
    
    # # Combine both conditions to create the final filter
    # filtered_df = df[positive_condition | negative_condition]
    
    # # Get the top n rows with the largest absolute daily returns
    # return filtered_df.nlargest(n, 'abs_daily_return')
    return df.nlargest(n, 'abs_daily_return')

# Define the API key and base URL
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

# Define the currency pairs and years
pair = "C:USDEUR"
years = range(2022, 2024)

# Initialize DataFrames
full_data = pd.DataFrame()
outliers_data = pd.DataFrame()

# Loop over each year
for year in years:
    # Format the API endpoint
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    url = f"https://api.polygon.io/v2/aggs/ticker/{pair}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
    
    # Make the API request
    response = requests.get(url)
    data = response.json()
    
    # Check if the request was successful
    if response.status_code == 200 and 'results' in data:
        # Load data into a DataFrame
        df = pd.DataFrame(data['results'])
        # Convert timestamps
        df['date'] = pd.to_datetime(df['t'], unit='ms')
        df.drop(columns=['t'], inplace=True)

        df = df[df['date'].dt.weekday < 5]
        # Calculating returns
        df = calculate_daily_return(df)

        # Append the data to the full_data DataFrame for the current currency pair
        df['year'] = year
        df['day'] = df['date'].dt.day_name()
        # Find the top 10 outliers based on absolute values of the daily return value
        # df['abs_daily_return'] = df['c'].abs()
        top_outliers = df.nlargest(10, 'abs_daily_return')
        outlier_dates = top_outliers['date']

        # Create a new column 'is_outlier' in the full_data DataFrame
        df['is_outlier'] = df['date'].isin(outlier_dates).astype(int)
        full_data = pd.concat([full_data, df], ignore_index=True)
        
        # Append outliers to the outliers_data DataFrame for the current currency pair
        top_outliers['year'] = year
        outliers_data = pd.concat([outliers_data, top_outliers], ignore_index=True)
        

sorted_full_data = full_data.sort_values(by="date")
sorted_outliers_data = outliers_data.sort_values(by="date")

In [3]:
sorted_full_data

,v,vw,o,c,h,l,n,date,daily_return,abs_daily_return,year,day,is_outlier
0,57901,0.8828,0.87925,0.88478,0.88651,0.879020,57901,2022-01-03,NaN,NaN,2022,Monday,0
1,62051,0.8854,0.88460,0.88624,0.88709,0.883000,62051,2022-01-04,0.001650,0.001650,2022,Tuesday,0
2,64819,0.8838,0.88627,0.88390,0.88673,0.881100,64819,2022-01-05,-0.002640,0.002640,2022,Wednesday,0
3,68551,0.8844,0.88409,0.88520,0.88613,0.882300,68551,2022-01-06,0.001471,0.001471,2022,Thursday,0
4,45968,0.8833,0.88531,0.88000,0.88571,0.879662,45968,2022-01-07,-0.005874,0.005874,2022,Friday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,6522,0.9078,0.90750,0.90747,0.90940,0.901600,6522,2023-12-25,0.000629,0.000629,2023,Monday,0
516,51232,0.9068,0.90747,0.90550,0.90833,0.905100,51232,2023-12-26,-0.002171,0.002171,2023,Tuesday,0
517,77389,0.9029,0.90557,0.90018,0.90670,0.898796,77389,2023-12-27,-0.005875,0.005875,2023,Wednesday,0
518,83788,0.9008,0.90018,0.90348,0.90453,0.897500,83788,2023-12-28,0.003666,0.003666,2023,Thursday,0


In [4]:
# Convert dates in dataset to datetime objects
sorted_outliers_data['date'] = pd.to_datetime(sorted_outliers_data['date'])

date_ranges = pd.DataFrame({
    "start_date": sorted_outliers_data['date'] - BDay(14), # To predict X days, keep this as X-1 (as 1 day of outlier will be considered in LSTM input)
    "end_date": sorted_outliers_data['date'] + BDay(15),
    "outlier_date": sorted_outliers_data['date'],
    "outlier_price": sorted_outliers_data['c'],
    "daily_return": sorted_outliers_data['daily_return']
})

date_ranges.reset_index(drop=True, inplace=True)

date_ranges

,start_date,end_date,outlier_date,outlier_price,daily_return
0,2022-02-17,2022-03-30,2022-03-09,0.90302,-0.015868
1,2022-06-15,2022-07-26,2022-07-05,0.97493,0.017163
2,2022-06-21,2022-08-01,2022-07-11,0.99537,0.014235
3,2022-08-24,2022-10-04,2022-09-13,1.00213,0.015134
4,2022-09-01,2022-10-12,2022-09-21,1.01711,0.014371
5,2022-09-05,2022-10-14,2022-09-23,1.03190,0.015290
6,2022-09-14,2022-10-25,2022-10-04,1.00150,-0.014950
7,2022-10-17,2022-11-25,2022-11-04,1.00150,-0.023575
8,2022-10-21,2022-12-01,2022-11-10,0.98115,-0.016913
9,2022-10-24,2022-12-02,2022-11-11,0.96400,-0.017479


In [5]:
def fetch_daily_data(pair, start_date, end_date, api_key):
    formatted_start_date = start_date.strftime('%Y-%m-%d')
    formatted_end_date = end_date.strftime('%Y-%m-%d')

    url = f"https://api.polygon.io/v2/aggs/ticker/{pair}/range/1/day/{formatted_start_date}/{formatted_end_date}?adjusted=true&sort=asc&apiKey={api_key}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code} - {response.text}")
        return None

    response_data = response.json()

    if 'results' not in response_data:
        print(f"No 'results' in response: {response_data}")
        return None

    df = pd.DataFrame(response_data['results'])
    df['date'] = pd.to_datetime(df['t'], unit='ms')
    df.drop(columns=['t'], inplace=True)

    daily_data = calculate_daily_return(df)
    daily_data.set_index('date', inplace=True)

    return daily_data

def fetch_and_process_daily_data(pair, start_date, end_date, api_key):
    daily_data = fetch_daily_data(pair, start_date, end_date, api_key)

    if daily_data is None:
        print("No data fetched")
        return None

    daily_data.reset_index(inplace=True)
    return daily_data

In [6]:
# Create an empty list to store the model performance metrics for each outlier
trade_results_list = []

# Define the profit threshold for entry (0.01% profit target)
profit_threshold = 0.1  # 0.1% in decimal form

# Loop through each row in the date_ranges DataFrame
for idx, row in date_ranges.iterrows():
    outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
    start_date_co = pd.Timestamp(row['start_date'])
    end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
    outlier_date_co = pd.Timestamp(row['outlier_date'])

    # Fetch and process daily data for the current range
    daily_data = fetch_and_process_daily_data(pair, start_date_co, end_date_co, api_key)

    if daily_data is None:
        print(f"No data fetched for outlier_id: {outlier_id}")
        continue  # Skip to the next iteration if no data

    # Assign the current outlier_id to the data
    daily_data['outlier_id'] = outlier_id

    # Filter out weekends
    daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

    # Sort data by date
    daily_data = daily_data.sort_values(by='date', ascending=True)

    # Fill missing values
    daily_data.fillna(method='bfill', inplace=True)
    daily_data.fillna(method='ffill', inplace=True)

    # Split the dataset into train and test sets
    train_set = daily_data.iloc[:15].reset_index(drop=True)
    test_set = daily_data.iloc[15:].reset_index(drop=True)

    # Normalize the data using only the training data
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_set[["c"]])

    # Prepare data for LSTM model
    sequence_length = 12
    train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

    # Define and compile LSTM model
    model = Sequential([
        LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

    # Fit the model
    model.fit(train_generator, epochs=100, verbose=0)

    # Prepare the last sequence for forecasting
    last_sequence = train_scaled[-sequence_length:]

    # Forecast the next steps
    forecast_steps = len(test_set)
    predictions_scaled = []
    for _ in range(forecast_steps):
        last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
        next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
        predictions_scaled.append(next_step_pred.ravel()[0])
        last_sequence = np.roll(last_sequence, -1)
        last_sequence[-1] = next_step_pred

    # Inverse transform predictions
    predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

    # Actual values for comparison
    actuals = test_set["c"].values[:forecast_steps]
    
    # Calculate MAPE (Mean Absolute Percentage Error)
    mse = mean_squared_error(actuals, predictions_inv)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(actuals, predictions_inv)
    accuracy = np.round(100 - (mape * 100), 2)

    # Determine if the outlier is positive or negative
    outlier_return = row['daily_return']
    is_positive_outlier = outlier_return > 0  # Check if the outlier is positive
    outlier_type = "Positive" if is_positive_outlier else "Negative"  # Define outlier type

    # Trade logic based on outlier type
    entry_price = None
    exit_price = None
    entry_date = None
    exit_date = None
    profit = 0
    days_held = 0
    trade_initiated = False
    position_type = "No Trade"  # Default if no trade is initiated

    for i in range(1, len(predictions_inv)):
        predicted_change = (predictions_inv[i] - predictions_inv[i - 1]) * 100 / predictions_inv[i - 1]
        actual_change = (actuals[i] - actuals[i - 1]) * 100 / actuals[i - 1]

        if is_positive_outlier:
            # Positive Outlier Logic (Long Position)
            # Trend continuation: Enter long position if predicted trend continues
            if entry_price is None and predicted_change >= profit_threshold:
                future_predictions = predictions_inv[i:i + 2]  # Check the next 2 predicted values
                if all((future_predictions[j] - future_predictions[j - 1]) * 100 / future_predictions[j - 1] >= profit_threshold for j in range(1, len(future_predictions))):
                    entry_price = actuals[i - 1]
                    entry_date = test_set['date'].iloc[i - 1]
                    trade_initiated = True
                    position_type = "Long"
            # Trend reversal: Exit long position if predicted change turns negative
            elif entry_price is not None and predicted_change < 0:
                exit_price = actuals[i - 1]
                exit_date = test_set['date'].iloc[i - 1]
                profit = (exit_price - entry_price) * 100 / entry_price
                days_held = (exit_date - entry_date).days
                break
        else:
            # Negative Outlier Logic (Short Position)
            # Trend continuation: Short the market if predicted trend continues
            if entry_price is None and predicted_change <= -profit_threshold:
                future_predictions = predictions_inv[i:i + 2]  # Check the next 2 predicted values
                if all((future_predictions[j] - future_predictions[j - 1]) * 100 / future_predictions[j - 1] <= -profit_threshold for j in range(1, len(future_predictions))):
                    entry_price = actuals[i - 1]
                    entry_date = test_set['date'].iloc[i - 1]
                    trade_initiated = True
                    position_type = "Short"
            # Trend reversal: Exit short position if predicted change turns positive
            elif entry_price is not None and predicted_change > 0:
                exit_price = actuals[i - 1]
                exit_date = test_set['date'].iloc[i - 1]
                profit = (entry_price - exit_price) * 100 / entry_price  # Short trade profit formula
                days_held = (exit_date - entry_date).days
                break

    # If no reversal is detected, exit at the last available date
    if trade_initiated and entry_price is not None and exit_price is None:
        exit_price = actuals[-1]
        exit_date = test_set['date'].iloc[-1]
        if is_positive_outlier:
            profit = (exit_price - entry_price) * 100 / entry_price  # Long trade profit formula
        else:
            profit = (entry_price - exit_price) * 100 / entry_price  # Short trade profit formula
        days_held = (exit_date - entry_date).days

    # Adjust "Position Type" based on whether a trade was initiated or not
    if not trade_initiated:
        if is_positive_outlier:
            position_type = "Go Short (Sell!)"
        else:
            position_type = "Go Long (Buy!)"

    # Append trade results for every outlier, even if no trade was initiated
    if profit >= 0 and rmse < 0.5:
        trade_results_list.append({
			'Outlier Date': outlier_date_co,
			'Outlier Price': row['outlier_price'],
			'Outlier Type': outlier_type,  # New column for outlier type
			'Position Type': position_type,  # Adjusted position type
			'Entry Date': entry_date,
			'Entry Price': entry_price,
			'Exit Date': exit_date,
			'Exit Price': exit_price,
			'Days Held': days_held if trade_initiated else 0,
			'Profit': np.round(profit, 2) if trade_initiated else None,
			'model_RMSE': rmse,
			'model_accuracy': accuracy
		})

# Convert the list of dictionaries to a DataFrame
trade_results_df = pd.DataFrame(trade_results_list)

/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_7339/1944401145.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_data.fillna(method='bfill', inplace=True)
/var/folders/9n/nyfs9h7n2lsfs0vd2lq0589h0000gn/T/ipykernel_7339/1944401145.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_data.fillna(method='ffill', inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120

In [7]:
trade_results_df

,Outlier Date,Outlier Price,Outlier Type,Position Type,Entry Date,Entry Price,Exit Date,Exit Price,Days Held,Profit,model_RMSE,model_accuracy
0,2022-09-21,1.01711,Positive,Go Short (Sell!),NaT,NaN,NaT,NaN,0,NaN,0.016195,98.64
1,2022-11-04,1.00150,Negative,Short,2022-11-15,0.96545,2022-11-17,0.96513,2,0.03,0.045054,95.69
2,2022-11-10,0.98115,Negative,Short,2022-11-11,0.96400,2022-11-16,0.96200,5,0.21,0.017652,98.41
3,2022-11-11,0.96400,Negative,Short,2022-11-14,0.96898,2022-12-02,0.94890,18,2.07,0.067754,94.27
4,2023-01-06,0.93760,Negative,Go Long (Buy!),NaT,NaN,NaT,NaN,0,NaN,0.021365,97.74
5,2023-02-01,0.90790,Negative,Go Long (Buy!),NaT,NaN,NaT,NaN,0,NaN,0.019498,98.04
6,2023-03-07,0.94780,Positive,Long,2023-03-13,0.93254,2023-03-15,0.94499,2,1.34,0.013603,98.82
7,2023-03-15,0.94499,Positive,Long,2023-03-22,0.92001,2023-03-28,0.92257,6,0.28,0.021360,97.80
8,2023-03-17,0.92990,Negative,Short,2023-03-24,0.92750,2023-03-28,0.92257,4,0.53,0.012827,98.74
9,2023-05-01,0.91164,Positive,Long,2023-05-22,0.92499,2023-05-23,0.92840,1,0.37,0.016727,98.40


### Model Performance

In [8]:
# # Create an empty list to store the model performance metrics for each outlier
# model_perf_list = []

# # Loop through each row in the date_ranges DataFrame
# for idx, row in date_ranges.iterrows():
#     outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(symbol, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {outlier_id}")
#         continue  # Skip to the next iteration if no data

#     # Assign the current outlier_id to the data
#     daily_data['outlier_id'] = outlier_id

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Split the dataset into train and test sets
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)

#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     sequence_length = 12
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]

#     # Calculate evaluation metrics
#     mse = mean_squared_error(actuals, predictions_inv)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(actuals, predictions_inv)
#     mape = mean_absolute_percentage_error(actuals, predictions_inv)

#     # Append the performance metrics to the list
#     model_perf_list.append({
#         'outlier_id': outlier_id,  # Each range will have a unique outlier_id starting from 1
#         'outlier_date': row['outlier_date'],
#         'MAE': mae,
#         'MSE': mse,
#         'RMSE': rmse,
#         'MAPE': mape
#     })

# # Convert the list of dictionaries to a DataFrame
# model_perf = pd.DataFrame(model_perf_list)

In [9]:
# model_perf

### MA of Predictions

In [10]:
# # Define the function to find entry and exit points based on moving averages
# def find_entry_exit(predictions, window_short=5, window_long=10):
    # """Find entry and exit points based on the moving average of predictions."""
#     # Calculate short-term and long-term moving averages
#     short_ma = predictions.rolling(window=window_short).mean()
#     long_ma = predictions.rolling(window=window_long).mean()

#     # Generate signals: 1 for buy, -1 for sell
#     buy_signal = (short_ma > long_ma) & (short_ma.shift(1) <= long_ma.shift(1))
#     sell_signal = (short_ma < long_ma) & (short_ma.shift(1) >= long_ma.shift(1))

#     return buy_signal, sell_signal

### MA of wavelegth of predictions

In [11]:
# def calculate_wavelength(predictions, wavelet='gaus2', scales=np.logspace(0.1, 1.5, num=20, base=10)):
#     """Calculate the wavelength using Continuous Wavelet Transform (CWT)."""
#     # Compute the Continuous Wavelet Transform
#     coefficients, _ = pywt.cwt(predictions, scales, wavelet)

#     # For simplicity, we use the mean of the coefficients across all scales as the wavelength feature
#     wavelength = np.mean(coefficients, axis=0)

#     return wavelength

# def find_entry_exit(predictions, window_short=5, window_long=10, wavelet='gaus2'):
#     """Find entry and exit points based on the wavelength of predictions."""
#     # Convert the predictions to their wavelength using CWT
#     wavelength = calculate_wavelength(predictions, wavelet)

#     # Convert to pandas Series for rolling window calculations
#     wavelength_series = pd.Series(wavelength)

#     # Calculate short-term and long-term moving averages on the wavelength
#     short_ma = wavelength_series.rolling(window=window_short).mean()
#     long_ma = wavelength_series.rolling(window=window_long).mean()

#     # Generate signals: 1 for buy, -1 for sell
#     buy_signal = (short_ma > long_ma) & (short_ma.shift(1) <= long_ma.shift(1))
#     sell_signal = (short_ma < long_ma) & (short_ma.shift(1) >= long_ma.shift(1))

#     return buy_signal, sell_signal


### RSI

In [12]:
# def calculate_rsi(prices, window=14):
#     """Calculate Relative Strength Index (RSI) for the given prices."""
#     delta = prices.diff(1)  # Price changes
#     gain = delta.where(delta > 0, 0)  # Positive gains
#     loss = -delta.where(delta < 0, 0)  # Negative losses

#     avg_gain = gain.rolling(window=window, min_periods=1).mean()
#     avg_loss = loss.rolling(window=window, min_periods=1).mean()

#     rs = avg_gain / avg_loss  # Relative Strength
#     rsi = 100 - (100 / (1 + rs))  # RSI Formula

#     return rsi

# def find_entry_exit(predictions, rsi_window=14, buy_threshold=40, sell_threshold=60):
#     """Find entry and exit points based on adjusted RSI thresholds."""
#     prices = pd.Series(predictions.ravel())  # Convert predictions to a Pandas Series

#     # Calculate RSI
#     rsi = calculate_rsi(prices, window=rsi_window)

#     # Generate buy (RSI < buy_threshold) and sell (RSI > sell_threshold) signals
#     buy_signal = (rsi < buy_threshold)
#     sell_signal = (rsi > sell_threshold)

#     return buy_signal, sell_signal



In [13]:
# # Create an empty list to store the model performance metrics for each outlier
# model_perf_list = []

# # Loop through each row in the date_ranges DataFrame
# for idx, row in date_ranges.iterrows():
#     outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(symbol, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {outlier_id}")
#         continue  # Skip to the next iteration if no data

#     # Assign the current outlier_id to the data
#     daily_data['outlier_id'] = outlier_id

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Split the dataset into train and test sets
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)

#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     sequence_length = 12
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]

#     # Add predictions to the test set for entry/exit calculation
#     test_set['predictions'] = predictions_inv

#     # Find entry and exit signals
#     buy_signal, sell_signal = find_entry_exit(pd.Series(predictions_inv.ravel()))

#     # Store the entry and exit dates
#     entry_date = test_set['date'][buy_signal].min() if buy_signal.any() else None
#     exit_date = test_set['date'][sell_signal].min() if sell_signal.any() else None

#     # Initialize profit/loss percentage to None if no trade is possible
#     profit_loss_percent = None

#     # If we have both entry and exit dates, calculate the profit or loss percentage
#     if entry_date is not None and exit_date is not None:
#         entry_price = test_set.loc[test_set['date'] == entry_date, 'predictions']
#         exit_price = test_set.loc[test_set['date'] == exit_date, 'predictions']

#         # Check if entry and exit prices exist (i.e., are not empty)
#         if not entry_price.empty and not exit_price.empty:
#             entry_price = entry_price.values[0]
#             exit_price = exit_price.values[0]
#             profit_loss_percent = ((exit_price - entry_price) / entry_price) * 100

#     # Calculate evaluation metrics
#     mse = mean_squared_error(actuals, predictions_inv)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(actuals, predictions_inv)
#     mape = mean_absolute_percentage_error(actuals, predictions_inv)

#     # Append the performance metrics to the list, including entry and exit dates and profit/loss percentage
#     model_perf_list.append({
#         'outlier_id': outlier_id,  # Each range will have a unique outlier_id starting from 1
#         'outlier_date': row['outlier_date'],
#         'MAE': mae,
#         'MSE': mse,
#         'RMSE': rmse,
#         'MAPE': mape,
#         'entry_date': entry_date,
#         'exit_date': exit_date,
#         'profit_loss_percent': profit_loss_percent
#     })

# # Convert the list of dictionaries to a DataFrame
# model_perf = pd.DataFrame(model_perf_list)

In [14]:
# model_perf

### Combined Strategy

In [15]:
# # Create an empty list to store the model performance metrics for each outlier
# model_perf_list = []

# # Loop through each row in the date_ranges DataFrame
# for idx, row in date_ranges.iterrows():
#     outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(symbol, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {outlier_id}")
#         continue  # Skip to the next iteration if no data

#     # Assign the current outlier_id to the data
#     daily_data['outlier_id'] = outlier_id

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Split the dataset into train and test sets
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)

#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     sequence_length = 12
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]

#     # Step 1: Detect a Trend Reversal (Checking if prices are declining for 5 consecutive days and then increasing)
#     trend_reversal_detected = False
#     entry_date = None
#     entry_price = None
#     exit_date = None
#     exit_price = None
#     volatility_threshold = 0.05  # Threshold for detecting a volatility spike
#     low_volatility = True

#     for i in range(5, forecast_steps):
#         # Calculate price changes
#         past_5_days_change = np.mean(predictions_inv[i-5:i]) - np.mean(predictions_inv[i-10:i-5])
#         future_change = np.mean(predictions_inv[i:i+5]) - np.mean(predictions_inv[i-5:i])

#         # Check for a trend reversal
#         if past_5_days_change < 0 and future_change > 0 and not trend_reversal_detected:
#             entry_date = test_set.iloc[i]['date']
#             entry_price = predictions_inv[i][0]
#             trend_reversal_detected = True
#             print(f"Trend reversal detected on {entry_date}")

#         # Step 2: Check for volatility (If volatility increases, it's a signal to exit)
#         if trend_reversal_detected:
#             volatility = np.std(predictions_inv[i:i+5])  # Volatility is measured by the standard deviation of the next 5 days
#             if volatility > volatility_threshold and low_volatility:
#                 exit_date = test_set.iloc[i]['date']
#                 exit_price = predictions_inv[i][0]
#                 print(f"Volatility spike detected on {exit_date}. Exiting trade.")
#                 break
#             low_volatility = volatility <= volatility_threshold

#     # Step 3: If no exit detected by volatility, exit at the last prediction
#     if exit_date is None:
#         exit_date = test_set.iloc[-1]['date']
#         exit_price = predictions_inv[-1][0]
#         print(f"No volatility spike detected. Exiting trade on {exit_date}")

#     # Calculate profit or loss
#     if entry_price is not None and exit_price is not None:
#         profit_loss = exit_price - entry_price
#     else:
#         profit_loss = None

#     # Calculate evaluation metrics
#     mse = mean_squared_error(actuals, predictions_inv)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(actuals, predictions_inv)
#     mape = mean_absolute_percentage_error(actuals, predictions_inv)

#     # Append the performance metrics to the list, including entry/exit dates and profit/loss
#     model_perf_list.append({
#         'outlier_id': outlier_id,
#         'outlier_date': row['outlier_date'],
#         'entry_date': entry_date,
#         'exit_date': exit_date,
#         'profit_loss': profit_loss,
#         'MAE': mae,
#         'MSE': mse,
#         'RMSE': rmse,
#         'MAPE': mape
#     })

# # Convert the list of dictionaries to a DataFrame
# model_perf = pd.DataFrame(model_perf_list)

In [16]:
# model_perf

### Combined Logic

In [17]:
# # Placeholder function to get LSTM predictions (integrating your LSTM logic)
# def get_lstm_prediction(daily_data, sequence_length=12):
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)
    
#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))
#     last_observed_price = daily_data['c'].values[-1]

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]

#     # Calculate evaluation metrics
#     mse = mean_squared_error(actuals, predictions_inv)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(actuals, predictions_inv)
#     mape = mean_absolute_percentage_error(actuals, predictions_inv)

#     return {
#         'predictions': predictions_inv,
#         'mse': mse,
#         'rmse': rmse,
#         'mae': mae,
#         'mape': mape,
#         'predicted_trend': 'up' if predictions_inv.mean() > last_observed_price else 'down',
#         'predicted_volatility': np.std(predictions_inv)
#     }

# # Function to adjust position size based on volatility
# def adjust_position_size(volatility):
#     if volatility < 0.2:
#         return 1.0  # Full position size for low volatility
#     elif 0.2 <= volatility <= 0.5:
#         return 0.75  # Moderate position size
#     else:
#         return 0.5  # Reduce position size for high volatility

In [18]:
# # Define tolerance level and max search days for recovery
# tolerance = 0.005
# max_days = 10

# # Create a DataFrame to store trade results
# trade_data = pd.DataFrame(columns=[
#     'Outlier Date', 'Entry Date', 'Entry Price', 'Exit Date', 'Exit Price', 'Days Held', 'Profit', 'Volatility Spike Date', 'Position Size'
# ])

# # Loop through each outlier in date_ranges
# for idx, row in date_ranges.iterrows():
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(symbol, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {idx + 1}")
#         continue  # Skip to the next iteration if no data

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Get LSTM predictions and forecast
#     lstm_result = get_lstm_prediction(daily_data)

#     if lstm_result['predicted_trend'] == 'up':
#         # Entry point: outlier date
#         entry_date = outlier_date_co
#         entry_price = daily_data.loc[daily_data['date'] == outlier_date_co, 'c'].values[0]
#         predicted_volatility = lstm_result['predicted_volatility']
#         position_size = adjust_position_size(predicted_volatility)

#         # Search for exit point based on volatility spikes or recovery within max_days
#         future_data = daily_data[daily_data['date'] > outlier_date_co].head(max_days)
#         exit_date = None
#         exit_price = None
#         days_held = 0
#         profit = 0
#         volatility_spike_date = None

#         for _, future_day in future_data.iterrows():
#             future_date = future_day['date']
#             future_close = future_day['c']
#             future_volatility = lstm_result['predicted_volatility']  # You can improve this with dynamic prediction

#             # Volatility spike detected
#             if future_volatility > 0.5:
#                 volatility_spike_date = future_date
#                 exit_date = future_date
#                 exit_price = future_close
#                 days_held = len(pd.bdate_range(start=entry_date, end=exit_date))
#                 profit = (exit_price - entry_price) * position_size
#                 break

#             # Check if recovery within tolerance
#             if np.abs(future_close - entry_price) <= tolerance * entry_price:
#                 exit_date = future_date
#                 exit_price = future_close
#                 days_held = len(pd.bdate_range(start=entry_date, end=exit_date))
#                 profit = (exit_price - entry_price) * position_size
#                 break

#         # Default exit if no volatility spike or recovery
#         if exit_date is None:
#             last_day = future_data.iloc[-1]
#             exit_date = last_day['date']
#             exit_price = last_day['c']
#             days_held = max_days
#             profit = (exit_price - entry_price) * position_size

#         # Store the result
#         trade_info = {
#             'Outlier Date': outlier_date_co,
#             'Entry Date': entry_date,
#             'Entry Price': entry_price,
#             'Exit Date': exit_date,
#             'Exit Price': exit_price,
#             'Days Held': days_held,
#             'Profit': profit,
#             'Volatility Spike Date': volatility_spike_date,
#             'Position Size': position_size
#         }
#         trade_data = pd.concat([trade_data, pd.DataFrame([trade_info])], ignore_index=True)

### Threshold Strategy

In [19]:
# # Create an empty list to store the model performance metrics for each outlier
# trade_results_list = []

# # Define the threshold for entry (0.05% profit target)
# profit_threshold = 0.0005  # 0.05% in decimal form

# # Loop through each row in the date_ranges DataFrame
# for idx, row in date_ranges.iterrows():
#     outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(pair, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {outlier_id}")
#         continue  # Skip to the next iteration if no data

#     # Assign the current outlier_id to the data
#     daily_data['outlier_id'] = outlier_id

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Split the dataset into train and test sets
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)

#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     sequence_length = 12
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]

#     # Trade logic to find entry and exit points
#     entry_price = None
#     exit_price = None
#     entry_date = None
#     exit_date = None
#     profit_loss = 0
#     days_held = 0

#     for i in range(1, len(predictions_inv)):
#         predicted_change = (predictions_inv[i] - predictions_inv[i - 1]) / predictions_inv[i - 1]
#         actual_change = (actuals[i] - actuals[i - 1]) / actuals[i - 1]

#         # Entry condition: Price predicted to increase by at least 0.05%
#         if entry_price is None and predicted_change >= profit_threshold:
#             entry_price = actuals[i - 1]
#             entry_date = test_set['date'].iloc[i - 1]
        
#         # Exit condition: Trend reversal (predicted change turns negative)
#         elif entry_price is not None and predicted_change < 0:
#             exit_price = actuals[i - 1]
#             exit_date = test_set['date'].iloc[i - 1]
#             profit_loss = (exit_price - entry_price) * 100 / entry_price  
#             days_held = (exit_date - entry_date).days
#             break
    
#     # If no reversal is detected, exit at the last available date
#     if entry_price is not None and exit_price is None:
#         exit_price = actuals[-1]
#         exit_date = test_set['date'].iloc[-1]
#         profit_loss = (exit_price - entry_price) * 100 / entry_price 
#         days_held = (exit_date - entry_date).days

#     # Append trade results if entry and exit occurred
#     if entry_price is not None and exit_price is not None:
#         trade_results_list.append({
#             'Outlier Date': outlier_date_co,
#             'Entry Date': entry_date,
#             'Entry Price': entry_price,
#             'Exit Date': exit_date,
#             'Exit Price': exit_price,
#             'Days Held': days_held,
#             'Profit/Loss': profit_loss
#         })

# # Convert the list of dictionaries to a DataFrame
# trade_results_df = pd.DataFrame(trade_results_list)

### Threshold Startegy 2.0

In [20]:
# # Create an empty list to store the model performance metrics for each outlier
# trade_results_list = []

# # Define the profit threshold for entry (0.01% profit target)
# profit_threshold = 0.1  # 0.1% in decimal form

# # Loop through each row in the date_ranges DataFrame
# for idx, row in date_ranges.iterrows():
#     outlier_id = idx + 1  # Assign a unique outlier_id for each iteration (starting with 1)
#     start_date_co = pd.Timestamp(row['start_date'])
#     end_date_co = pd.Timestamp(row['end_date']) + pd.Timedelta(days=1)
#     outlier_date_co = pd.Timestamp(row['outlier_date'])

#     # Fetch and process daily data for the current range
#     daily_data = fetch_and_process_daily_data(pair, start_date_co, end_date_co, api_key)

#     if daily_data is None:
#         print(f"No data fetched for outlier_id: {outlier_id}")
#         continue  # Skip to the next iteration if no data

#     # Assign the current outlier_id to the data
#     daily_data['outlier_id'] = outlier_id

#     # Filter out weekends
#     daily_data = daily_data[~daily_data['date'].dt.weekday.isin([5, 6])]

#     # Sort data by date
#     daily_data = daily_data.sort_values(by='date', ascending=True)

#     # Fill missing values
#     daily_data.fillna(method='bfill', inplace=True)
#     daily_data.fillna(method='ffill', inplace=True)

#     # Split the dataset into train and test sets
#     train_set = daily_data.iloc[:15].reset_index(drop=True)
#     test_set = daily_data.iloc[15:].reset_index(drop=True)

#     # Normalize the data using only the training data
#     scaler = MinMaxScaler()
#     train_scaled = scaler.fit_transform(train_set[["c"]])

#     # Prepare data for LSTM model
#     sequence_length = 12
#     train_generator = TimeseriesGenerator(train_scaled, train_scaled, length=sequence_length, batch_size=1)

#     # Define and compile LSTM model
#     model = Sequential([
#         LSTM(64, activation='relu', input_shape=(sequence_length, 1)),
#         Dense(1)
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

#     # Fit the model
#     model.fit(train_generator, epochs=100, verbose=0)

#     # Prepare the last sequence for forecasting
#     last_sequence = train_scaled[-sequence_length:]

#     # Forecast the next steps
#     forecast_steps = len(test_set)
#     predictions_scaled = []
#     for _ in range(forecast_steps):
#         last_sequence_reshaped = last_sequence.reshape((1, sequence_length, 1))
#         next_step_pred = model.predict(last_sequence_reshaped, verbose=0)
#         predictions_scaled.append(next_step_pred.ravel()[0])
#         last_sequence = np.roll(last_sequence, -1)
#         last_sequence[-1] = next_step_pred

#     # Inverse transform predictions
#     predictions_inv = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

#     # Actual values for comparison
#     actuals = test_set["c"].values[:forecast_steps]
    
# 	# Calculate MAPE (Mean Absolute Percentage Error)
#     mse = mean_squared_error(actuals, predictions_inv)
#     rmse = np.sqrt(mse)
#     mape = mean_absolute_percentage_error(actuals, predictions_inv)
#     accuracy = np.round(100 - (mape * 100), 2)

#     # Trade logic to find entry and exit points
#     entry_price = None
#     exit_price = None
#     entry_date = None
#     exit_date = None
#     profit_loss = 0
#     days_held = 0
#     trade_initiated = False

#     for i in range(1, len(predictions_inv)):
#         predicted_change = (predictions_inv[i] - predictions_inv[i - 1]) * 100 / predictions_inv[i - 1]
#         actual_change = (actuals[i] - actuals[i - 1]) * 100 / actuals[i - 1]

#         # Entry condition: Price predicted to increase by at least 0.05% consistently for a few days
#         if entry_price is None and predicted_change >= profit_threshold:
#             # Check if future predictions show a continued upward trend
#             future_predictions = predictions_inv[i:i + 2]  # Look at the next 3 predicted values
#             if all((future_predictions[j] - future_predictions[j - 1]) * 100 / future_predictions[j - 1] >= profit_threshold for j in range(1, len(future_predictions))):
#                 entry_price = actuals[i - 1]
#                 entry_date = test_set['date'].iloc[i - 1]
#                 trade_initiated = True
#             else:
#                 trade_initiated = False  # Skip trade if no consistent upward trend

#         # Exit condition: Trend reversal (predicted change turns negative)
#         elif entry_price is not None and predicted_change < 0:
#             exit_price = actuals[i - 1]
#             exit_date = test_set['date'].iloc[i - 1]
#             profit_loss = (exit_price - entry_price) * 100 / entry_price
#             days_held = (exit_date - entry_date).days
#             break

#     # If no reversal is detected, exit at the last available date
#     if trade_initiated and entry_price is not None and exit_price is None:
#         exit_price = actuals[-1]
#         exit_date = test_set['date'].iloc[-1]
#         profit_loss = (exit_price - entry_price) * 100 / entry_price
#         days_held = (exit_date - entry_date).days

#     # Append trade results only if there was a valid trade (i.e., entry and exit with profit > 0.1%) and rmse < 0.05
#     if trade_initiated and entry_price is not None and exit_price is not None and profit_loss > profit_threshold and rmse < 0.0:
#         trade_results_list.append({
#             'Outlier Date': outlier_date_co,
#             'Outlier Price': row['outlier_price'],
#             'Entry Date': entry_date,
#             'Entry Price': entry_price,
#             'Exit Date': exit_date,
#             'Exit Price': exit_price,
#             'Days Held': days_held,
#             'Profit': profit_loss,
#             'model_RMSE': rmse,
#             'model_accuracy': accuracy
#         })

# # Convert the list of dictionaries to a DataFrame
# trade_results_df = pd.DataFrame(trade_results_list)

# # Show the final dataframe with trade results
# trade_results_df